In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d  emmarex/plantdisease

In [ ]:
! unzip plantdisease.zip

In [ ]:
import os
import numpy as np
import shutil
rootdir= '/content/plantvillage/PlantVillage' #path of the original folder

classes = ['Pepper__bell___Bacterial_spot','Pepper__bell___healthy',
           'Potato___Early_blight','Potato___Late_blight','Potato___healthy',
           'Tomato_Bacterial_spot', 'Tomato_Early_blight','Tomato_Late_blight',
           'Tomato_Leaf_Mold','Tomato_Septoria_leaf_spot',
           'Tomato_Spider_mites_Two_spotted_spider_mite','Tomato__Target_Spot',
           'Tomato__Tomato_YellowLeaf__Curl_Virus','Tomato__Tomato_mosaic_virus','Tomato_healthy']

for i in classes:
  os.makedirs(rootdir +'/train/' + i)
  os.makedirs(rootdir +'/valid/' + i)
  source = rootdir + '/' + i
  allFileNames = os.listdir(source)
  np.random.shuffle(allFileNames)
  test_ratio = 0.25
  train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                      [int(len(allFileNames)* (1 - test_ratio))])
  train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

  for name in train_FileNames:
    shutil.copy(name, rootdir +'/train/' + i)

  for name in test_FileNames:
    shutil.copy(name, rootdir +'/valid/' + i)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [ ]:
os.listdir('/content/plantvillage/PlantVillage/train')

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.5,
                                  shear_range = 0.3,
                                  horizontal_flip =True,
                                  preprocessing_function = preprocess_input)
val_datagen = ImageDataGenerator( preprocessing_function =preprocess_input)

In [ ]:
train = train_datagen.flow_from_directory(directory="/content/plantvillage/PlantVillage/train",
                                          target_size =(256,256), batch_size =32)

val= val_datagen.flow_from_directory(directory="/content/plantvillage/PlantVillage/valid",
                                     target_size =(256,256), batch_size =32)

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False
  

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)

X = Dense(units = 15, activation= 'softmax')(X)

model = Model(base_model.input, X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor ='val_accuracy', min_delta= 0.01, patience = 3, verbose =1)

mc = ModelCheckpoint(filepath="best_model.h5",
                      monitor = "val_accuracy",
                      min_delta= 0.01, 
                      patience =3, 
                      verbose =1, 
                      save_best_only= True )

cb = [es, mc]

In [ ]:
his = model.fit_generator(train, 
                          steps_per_epoch =16,
                          epochs = 4,
                          verbose =1,
                          callbacks= cb,
                          validation_data =val,
                          validation_steps =16 )

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')
plt.title("loss vs v-loss")
plt.show()

In [ ]:
from keras.models import load_model
model = load_model("/content/best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]

print(f"accuracy of your model is = {acc*100} %")

In [ ]:
ref = dict(zip(list(train.class_indices.values()),   list(train.class_indices.keys())))

In [ ]:
def prediction(path):
  img = load_img(path, target_size=(256,256))
  i = img_to_array(img)
  im = preprocess_input(i)
  img = np.expand_dims(im,axis=0)
  pred = np.argmax(model.predict(img))
  print(f"The image belongs to {ref[pred]} ")

In [ ]:
path = "/content/plantvillage/PlantVillage/Potato___Late_blight/0051e5e8-d1c4-4a84-bf3a-a426cdad6285___RS_LB 4640.JPG"

prediction(path)

In [ ]:
import tensorflow as tf

In [ ]:
new_model = tf.keras.models.load_model('/content/best_model.h5')

# Check its architecture
new_model.summary()

In [ ]:
new_model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])